## Markov sampling


In [10]:
import numpy as np
import copy
import random

#### 1. Compute $Q(t)$ for a given partition

In [ ]:

###
# code from Mattia Gastoldi
###

def computeProb(yi, t, Ns, X_bar, mu0, v, m):
    
    # Update parameters
    Nsr = Ns.copy()
    Nsr[t] += 1
 
    # Update means
    X_barsr = X_bar.copy()
    X_barsr[t] = (Ns[t]*X_bar[t] + yi) / Nsr[t]
    
    # Compute Q[it]
    a = Ns[t] / (m * np.sqrt(v))
    b = (1 - v)/2
    
    c = 0
    for i in range(len(Nsr) - 1):
        c += Nsr[i] * (X_barsr - mu0)**2
        
    d = 0
    for i in range(len(Ns)):
        d += Ns[i] * (X_bar - mu0)**2
    
    return a * np.exp(b * (c - d))


def gaussianJumps(y: np.array, rho: list, i: int, mu0: float, v: float, m: float) -> np.array:
        """
        Compute the Q(t), with t in {1, ..., b + 1}
        
        Recive a partition rho, without the i-th observation. Compute the
        value of Q(t) according Crowley [1997, Section 5.3]
        
        
        Parameters
        ----------
        y : np.array
            Array of target variables
        
        rho : list
            List of the sets componing the partition
        
        i : int
            Index of the i-th observation
            
        mu0 : float
            Prior mean
        
        v : float
            Prior variance information
        
        m: float
            Prior number of combination
            
        
        Returns
        -------
        Q : np.array
            Output array.
        """
        
        # Add a void set in the end
        rho.append(set())
        
        # Compute dimentions and means
        Ns = np.zeros(len(rho))
        X_bar = np.zeros(len(rho))
        
        for it, sets in enumerate(rho):
            Ns[it] = len(sets)
            
            if len(sets) != 0:
                X_bar[it] = y[list(sets)].mean()
        
        # Compute Q(t)        
        Q = np.zeros(len(rho))
        for it in range(len(Q)-1):
           Q[it] = computeProb(y[i], rho, it, Ns, X_bar, mu0, v, m)
        Q[-1] = 1
             
            
        return Q/sum(Q)

### Markov sampling


In [2]:
# fake function to compute Q for debugging purposes
def compute_Qs(y: np.array, rho: list, i: int, mu0: float, v: float, m: float) -> np.array:

    rho.append({})

    Qs = np.ones(len(rho))

    return Qs

In [37]:
# model parameters :
mu0 = ...       #
sigma0 = ...    # 
m = ...         #


def Markov_step(current_partition, X, mu0, v, m):
    """
    Performs a single step of the markov chain. One step is defined as moving each element one time.
    
    Args:
        current_partition: list of sets of int, current partition of the data [{int,...,int},...,{int,...,int}]
        X: 1D array of float, observations associated to the objects
        mu0: float, model parameter
        v: float, model parameter
        m: float, model parameter
    Returns:
        new_partition: new partition chosen following the transition probabilities of the chain
    """
    print(current_partition)
    n = len(X)

    for i in range(n): # one step 
        # 1. Find in which cluster is i
        j = 0
        for index in range(len(current_partition)): # i is the index of the cluster
            if i in current_partition[index]:
                j = index
                break
        cluster = current_partition[j]

        # 2. Remove i from partition
        reduced_partition = copy.deepcopy(current_partition)

        if len(cluster) == 1: # case 1 : i is only element of cluster -> remove cluster
            del reduced_partition[j]
        
        else: # case 2 : cluster has more than 1 element -> remove i from cluster
            reduced_partition[j].remove(i) 

        # 3. Compute the transition weights
        temp = copy.deepcopy(reduced_partition) # just in case compute_Qs modifies the partition
        Qs = compute_Qs(X, temp, i, mu0, v, m)
        
        transitions = list(range(len(Qs))) # transitions are the index of the cluster where to move i.

        # randomly select a transition and compute new partition
        transition = random.choices(transitions, weights=Qs)[0]    # choose which transition to do based on weights
        if transition == (len(Qs) - 1): # i is added as new cluster {i}
            reduced_partition.append({i})
        
        else:   # add i to
            reduced_partition[transition].add(i)
        current_partition = reduced_partition
    # every element has been moved once
    return current_partition
    

def Markov_sampling(X, N, mu0, sigma0, m):
    """
    Performs N iterations of Markov sampling

    Args:
        X: array, observations
        N: int, number of iterations in of the markov sampling
        mu0: float, model parameter
        sigma0: float, model parameter
        m: float, model parameter
    """
    n = len(X)
    v = 1 / (n * sigma0**2)

    # initial partition, all objects are in their own partition
    inital_partition = []
    for i in range(n):
        inital_partition.append({i})

    samples = [inital_partition]    # for now we consider samples as the whole partition, could modify this later
    
    current_partition = inital_partition
    for step in range(N-1):
        current_partition = Markov_step(current_partition, X, mu0, v, m)
        samples.append(current_partition)

    return samples

[{0}, {1}, {2}, {3}]
[{2, 3}, {0, 1}]
[{0, 2, 3}, {1}]
[{0}, {1, 3}, {2}]
[{0, 1, 3}, {2}]
[{0, 3}, {1, 2}]
[{2}, {0, 3}, {1}]
[{0}, {1, 2}, {3}]
[{1}, {0, 2}, {3}]
[{0}, {1}, {2, 3}]
[{0}, {1, 2, 3}]
[{0, 1}, {2}, {3}]
[{0, 2, 3}, {1}]
[{2}, {0, 3}, {1}]
[{0, 2}, {1}, {3}]
[{0, 1, 2, 3}]
[{0, 3}, {1}, {2}]
[{0}, {1}, {2}, {3}]
[{0, 2}, {1}, {3}]
[{0, 2, 3}, {1}]
[{0, 1, 2}, {3}]
[{0}, {1, 2, 3}]
[{0}, {1}, {2, 3}]
[{0, 1, 2}, {3}]
[{1}, {0, 2, 3}]
[{0, 3}, {2}, {1}]
[{0, 1, 2}, {3}]
[{1}, {0, 2, 3}]
[{0, 3}, {1}, {2}]
[{0, 1}, {2}, {3}]
[{1, 3}, {0, 2}]
[{0, 1, 2}, {3}]
[{0, 1, 3}, {2}]
[{0, 1, 2, 3}]
[{0, 1, 2}, {3}]
[{2}, {0}, {1, 3}]
[{0, 3}, {1}, {2}]
[{0, 1}, {2}, {3}]
[{0, 2}, {1, 3}]
[{0}, {1}, {2, 3}]
[{0, 1}, {2}, {3}]
[{0}, {1, 2, 3}]
[{0, 1, 2, 3}]
[{0, 1, 3}, {2}]
[{0, 1, 2, 3}]
[{0, 3}, {1, 2}]
[{0, 3}, {1, 2}]
[{1, 2}, {0}, {3}]
[{0}, {1, 2, 3}]
[{0, 1}, {2}, {3}]
[{0}, {2, 3}, {1}]
[{1}, {0, 3}, {2}]
[{0, 3}, {2}, {1}]
[{2, 3}, {1}, {0}]
[{2}, {0, 1, 3}]
[{0}, {1, 3}, {

In [ ]:
X = np.array([1, 2, 2, 3])

result = Markov_sampling(X, 1000, 1, 1, 1)
print(result)